Gen List

In [94]:
import pandas as pd
from utilsData import fuelDict

df = pd.read_excel('data/Refrence/2023_celt_report.xlsx', skiprows=13, index_col=None, sheet_name='2.1 Generator List')
df2 = df[['RESOURCE ID', 'RESOURCE NAME', 'GENERATOR TYPE', 'PRIM FUEL TYPE', 'NAMEPLATE\n(MW)']]
df2.rename(columns={"NAMEPLATE\n(MW)": "Nameplate Capacity (MW)", "PRIM FUEL TYPE": "Primary Fuel Type", "RESOURCE ID": 'ID'}, inplace=True)


fuels = df2['Primary Fuel Type'].map(fuelDict)
# To add Pump Storage
# if True:
#     fuels.loc[(df2['Primary Fuel Type'] == 'WAT') & (df2['GENERATOR TYPE'] == 'HYDRO: PUMPED STORAGE')] = 'PS'

fuels = fuels.fillna('Other')
df2.insert(3, 'Fuel Type', fuels)

In [95]:
ID_celt = df2[['ID']]
ID_celt.dropna(inplace=True)

CSO

In [96]:
import pandas as pd
from datetime import datetime

In [97]:
df = pd.read_excel('data/Refrence/fca_obligations.xlsx', skiprows=0, index_col=None, sheet_name='FCA 14')
df = df.drop(['Intermittent', 'DR Type', 'Active/ Passive', 'Interface ID', 'Interface Name', 'Dispatch Zone ID', 'Dispatch Zone Name',
              'Lead Participant ID', 'Lead Participant Name', 'De-list Bid Type', 'De-list Bid MW', 'Status', 'Capacity Zone ID', 'Capacity Zone Name', 'Load Zone'], axis=1)
new_columns = pd.to_datetime(df.columns[-12:]).strftime('%B')
df.columns = df.columns[:-12].tolist() + new_columns.tolist()
df = df[df['Type'] == 'Generator']
df.reset_index(drop=True, inplace=True)


In [98]:
df = df2.merge(df[['ID', 'June', 'July', 'August', 'September', 'October', 'November', 'December', 'January', 'February', 'March', 'April', 'May']] , on=['ID'])

In [113]:
indices = (df.iloc[:, 5:6].to_numpy() < df.iloc[:, 6:].to_numpy()).any(axis=1)
tmp = df.iloc[indices, 6:].max(axis=1) * 1.05
df.iloc[indices, 5] = tmp.to_list()
(df.iloc[:, 5:6].to_numpy() >= df.iloc[:, 6:].to_numpy()).all(axis=1).all()

True

In [114]:
df = df.drop_duplicates(subset=['ID'], keep='first')

In [115]:
df.to_csv('data/generation.csv', index=False)

In [19]:
# from utilsData import getISO
# from utilsData import fuelDict
# dfISO, numGenerators, totalCap, totalCSO = getISO()

In [20]:
# def map(x):
#     # if 'biomass' in x:
#     #     print(x.lower())
#     x = x.lower()
#     if 'water' in x or 'hydro' in x:
#         return 'Hydro'
    
#     elif 'wind' in x:
#         return 'Wind'
    
#     elif 'waste' in x or 'wood' in x:
#         return 'Refuse/Woods'
    
#     elif 'solar' in x or 'photo' in x:
#         return 'Solar'
    
#     elif 'landfill gas' in x:
#         return 'Landfill Gas'
    
#     elif 'biomass gas' in x or 'methane' in x:
#         return 'Gas-Other'
    
#     elif 'gas' in x or 'ker' in x:
#         return 'Gas'
    
#     elif 'coal' in x:
#         return 'Coal'
    
#     elif 'oil' in x or 'diesel' in x or 'jet' in x:
#         return 'Oil'    
    
#     elif 'nuclear' in x:
#         return 'NUC'
    
#     elif 'storage' in x:
#         return 'ES'
    
#     else:
#         return 'Other'


In [21]:
# np.isin(df['ID'], dfISO['ASSET ID'])

In [22]:
# fuels = df[['ID', 'Generating Fuel Type']]
# fuels['Fuel Type'] = pd.Series()
# for i in range(0, len(fuels['Fuel Type'])):
#      a = dfISO[dfISO['ASSET ID'] == fuels.iloc[i]['ID']]['Fuel Type']
#      if len(a) > 0:
#           fuels['Fuel Type'][i] = a.to_list()[0]
#      else:
#           fuels['Fuel Type'][i] = map(fuels['Generating Fuel Type'][i])

In [23]:
# fuels = fuels.fillna('Other')
# set(fuels['Fuel Type'])

In [24]:
# df.insert(3, 'Fuel Type', fuels['Fuel Type'])
# df.to_csv('data/gen.csv', index=False)